## 🧹 Homogeneous Pure Shear Problem (Shear Stress Loading)

Solve a **homogeneous pure shear problem** by applying **shear stress (Neumann BC)** to the top boundaries of a square elastic block, fixed the bottom boundary, and compare the computed displacement field to the analytical solution:

$$u_x(x, y) = \gamma \cdot y$$

$$u_y(x, y) = 0 $$

$$\gamma = \frac{\tau}{G}$$


---

### 📌 Problem Description

- A **square elastic body** of size $L \times L$ is subjected to **uniform shear stress** $\tau_0$  
  on the **top boundary**: $y = L$.
- The **bottom boundary** ($y = 0$) is **fixed**.

---

### 🧱 Boundary Conditions

- **Left and right boundaries** ($x = 0, L$) :  
  Fixed vertical displacement 
  $$u_y(x, y) = 0$$
- **Bottom boundary** ($y = 0$):  
  Totally fixed 
- **Top boundaries** ($y = L$):  
  Shear Displacement: 
  $$u_x(x, y) = \Delta$$

  $$u_y(x, y) = 0$$




In [1]:
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
# for saving files later
tutorials_dir = Path.cwd()

In [3]:
# FEA problem info
ele_type = "D2_nn6_tri"
ndof = 2

In [4]:
# Define domain
L = 10.0      # length of the square plate
# number of elements along a direction, keep this an even number if you want the analytical solution to be able to compute midline deformation
nx = 5

# Define material properties
material_props = np.array([134.6, 83.33])  # [mu, K]
gamma = 0.1 # shear strain
Delta = gamma * L

# define the analytical solution
def analytical_solution(y):
    return gamma * y

In [5]:
# Generate mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, L, nx, nx)

In [6]:
# Plot mesh
mesh_img_fname = tutorials_dir / "pure_shear_mesh.png"
pre_demo.plot_mesh_2D(str(mesh_img_fname), ele_type, coords, connect)

<img src="pure_shear_mesh.png" alt="Mesh plot" style="width:50%;">

In [7]:
# Identify corner nodes
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0, L, 0, L)

In [8]:
# Apply boundary conditions:
# 1. Fix bottom boundary: u_x = 0 at y = 0.
fixed_bottom = pre.assign_fixed_nodes_rect(boundary_nodes, "bottom", 0.0, 0.0)
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", dof_1_disp=0.0)
fixed_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", dof_1_disp=0.0)
# 2. Prescribe right boundary: u_x = 0.5 * gamma 
fixed_top = pre.assign_fixed_nodes_rect(boundary_nodes, "top", Delta, 0.0)
fixed_nodes = np.hstack((fixed_bottom, fixed_top,fixed_left,fixed_right))

In [9]:
# No distributed load is applied
dload_info = np.empty((ndof + 2, 0))

In [10]:
# Number of incremental loading steps
nr_num_steps = 5

# Run the solver
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # solver expects coords as (ncoord, n_nodes)
    connect.T,     # and connectivity as (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-8,
    nr_maxit=30,
)

Step 0, load factor = 0.200
Iteration 1, Correction=1.000000e+00, Residual=2.741012e-03, tolerance=1.000000e-08
Iteration 2, Correction=1.113573e-03, Residual=3.425293e-04, tolerance=1.000000e-08
Iteration 3, Correction=4.247334e-08, Residual=1.956763e-08, tolerance=1.000000e-08
Iteration 4, Correction=7.281953e-16, Residual=1.725262e-15, tolerance=1.000000e-08
Step 1, load factor = 0.400
Iteration 1, Correction=5.000003e-01, Residual=2.741012e-03, tolerance=1.000000e-08
Iteration 2, Correction=5.566087e-04, Residual=3.427231e-04, tolerance=1.000000e-08
Iteration 3, Correction=2.123062e-08, Residual=1.960102e-08, tolerance=1.000000e-08
Iteration 4, Correction=3.450321e-16, Residual=2.088658e-15, tolerance=1.000000e-08
Step 2, load factor = 0.600
Iteration 1, Correction=3.333350e-01, Residual=2.741012e-03, tolerance=1.000000e-08
Iteration 2, Correction=3.708829e-04, Residual=3.428340e-04, tolerance=1.000000e-08
Iteration 3, Correction=1.414286e-08, Residual=1.962908e-08, tolerance=1.000

In [11]:
final_disp = displacements_all[-1]  # final global displacement vector (length = n_nodes * ndof)

In [12]:
# Extract nodes near center to get a 1D vertical slice.
tol_y = L / 20.0  # tolerance for y coordinate
mid_nodes = [i for i in range(coords.shape[0]) if abs(coords[i, 0] - L/2) < tol_y]
mid_nodes = sorted(mid_nodes, key=lambda i: coords[i, 1])  # sort by y-coordinate

In [13]:
# Extract y-coordinates and computed u_x from the final displacement.
y_vals = np.array([coords[i, 1] for i in mid_nodes])
computed_u_x = np.array([final_disp[ndof * i] for i in mid_nodes])
computed_u_y = np.array([final_disp[ndof * i+1] for i in mid_nodes])
analytical_u_x = analytical_solution(y_vals)

In [14]:
# Plot the computed and analytical u_x vs. x.
plt.figure(figsize=(8, 6))
plt.plot(y_vals, computed_u_x, 'ro-', label="Computed u_x")
plt.plot(y_vals, analytical_u_x, 'b--', label="Analytical u_x")
plt.xlabel("x (m)")
plt.ylabel("u_x (m)")
plt.title("Comparison of u_x(x): Computed vs. Analytical")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig('result_error.png', dpi=300, bbox_inches='tight')

<img src="result_error.png" alt="Mesh plot2" style="width:50%;">